# Combine subregional rebalanced zonation 4 runs and zonation 5 runs that don't need rebalancing

This combines the rebalanced subregional zonation 4 results and the marine and Caribbean zonation 5 results into one raster

Created by Amy Keister. Last run by Amy Keister on 8/14/2023. Took 2 hours to run.

In [11]:
import os, fnmatch
import arcpy

In [12]:
import time
start = time.time()

In [13]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
SourceWorkspace= r"D:\SE_Blueprint_2023\7_CombineZonation"
OutWorkspace = r"D:\SE_Blueprint_2023\7_CombineZonation\2_Combine.gdb"

In [14]:
OutTif = r"D:\SE_Blueprint_2023\7_CombineZonation\CombineZonation.tif"

In [15]:
# full SE Blueprint extent
Rextent= r"D:\SE_Blueprint_2023\2_FullExtent\SEBlueprint2023ExtentVIPR6.tif"

In [16]:
# define additional inputs
Marine= r"D:\SE_Blueprint_2023\6_ZonationRuns\Zonation\Caribbean_2023\rankmap.tif"
Car= r"D:\SE_Blueprint_2023\6_ZonationRuns\Zonation\Marine_2023\Marine_2023_CCandAtlEstXpoint1_NoSubCoreRule3to7\rankmap.tif"

### Start Analysis

In [17]:
# Set the source workspace
arcpy.env.workspace = SourceWorkspace

In [18]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\7_CombineZonation


## Rebalanced Zonation Results (Inland Subregions)

In [19]:
# Get and print a list of the rebalanced zonation results
rasters = arcpy.ListRasters("*", "TIF")
for raster in rasters:
    print(raster)

Z_Atlantic_Coastal_Plain_2023.tif
Z_Central_Gulf_Coastal_Plain_2023.tif
Z_Chihuahuan_Deserts_2023.tif
Z_East_Gulf_Coastal_Plain_Sub_2023.tif
Z_Edwards_Plateau_2023.tif
Z_Florida_Peninsula_2023.tif
Z_Gulf_Coastal_Prairies_2023.tif
Z_High_Plains_and_Tablelands_2023.tif
Z_Interior_Plateau_2023.tif
Z_Mid_East_Gulf_Coastal_Plain_2023.tif
Z_Mississippi_Alluvial_Valley_RZ_2023.tif
Z_North_Appalachians_2023.tif
Z_North_Missouri_2023.tif
Z_Ouachita_2023.tif
Z_Ozarks_and_Plains.tif
Z_Piedmont_2023.tif
Z_Plains_and_Timbers_2023.tif
Z_South_Appalachians_2023.tif
Z_South_Texas_Plains_2023.tif
Z_Texas_Blackland_Prairies_2023.tif
Z_West_Gulf_Coastal_Plain_2023.tif


In [20]:
# mosaic the rebalanced zonation results
arcpy.management.MosaicToNewRaster(rasters, OutWorkspace, "Mosaic1", None, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'D:\\SE_Blueprint_2023\\7_CombineZonation\\2_Combine.gdb\\Mosaic1'>

## Unbalanced Zonation Results (Marine and Caribbean Subregions)

In [21]:
# Set the workspace where outputs go
arcpy.env.workspace = OutWorkspace

### Stretch, round, and convert to integer

In [22]:
# take times 100
out_raster = arcpy.sa.Times(Marine, 100); out_raster.save("Marine100")
out_raster = arcpy.sa.Times(Car, 100); out_raster.save("Car100")

In [23]:
# Convert to integer
out_raster = arcpy.sa.Int("Marine100"); out_raster.save("Marine100_Int")
out_raster = arcpy.sa.Int("Car100"); out_raster.save("Car100_Int")

In [24]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\7_CombineZonation\2_Combine.gdb


## Combine inland and marine

In [25]:
# mosaic subregions to new raster
arcpy.management.MosaicToNewRaster("Mosaic1;Marine100_Int;Car100_Int", OutWorkspace, "Mosaic3", None, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'D:\\SE_Blueprint_2023\\7_CombineZonation\\2_Combine.gdb\\Mosaic3'>

In [36]:
# take times 1 to see if it fixes the striping
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Times("Mosaic3", 1); out_raster.save("Mosaic4")

In [37]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("Mosaic4", OutTif, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [28]:
end = time.time()
print(end - start)

6904.8453776836395
